In [1]:
import re
import sys
import os
import csv
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string
import operator

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

data_dir = "C:/Users/Rhea/Documents/INNOVATION/PROJECT/CLUSTER/"

In [2]:
def to_lower(text_list):
    text_list = [str(w).lower() for w in text_list]
    return text_list

In [3]:
def tokenize_docs(text_list):
    tokenized_docs = [word_tokenize(doc) for doc in text_list]
    return tokenized_docs

In [4]:
def remove_punctuation(tokenised_text):
    str1=string.punctuation
    str1=str1.replace('/','')
    regex = re.compile('[%s]' % re.escape(str1)) #see documentation here: http://docs.python.org/2/library/string.html

    tokenized_docs_no_punctuation = []

    for review in tokenised_text:
        
        new_review = []
        for token in review: 
            new_token = regex.sub(u'', token)
            if not new_token == u'':
                new_review.append(new_token)
        
        tokenized_docs_no_punctuation.append(new_review) 
    return tokenized_docs_no_punctuation

In [28]:
def remove_stopwords(tokenised_text):
    tokenized_docs_no_stopwords = []
    for doc in tokenised_text:
        new_term_vector = []
        for word in doc:
            if not word in stopwords.words('english') and not word in ['phone','apple','samsung','amazon','blu','iphone']:
                new_term_vector.append(word)
        tokenized_docs_no_stopwords.append(new_term_vector)
    return tokenized_docs_no_stopwords       


In [6]:
def lemmatise_tokens(tokenized_text):
    wordnet = WordNetLemmatizer()
    lemmatized_docs = []
    for doc in tokenized_text:
        final_doc = []
        for word in doc:
            final_doc.append(wordnet.lemmatize(word))
            lemmatized_docs.append(final_doc)
    return lemmatized_docs

In [7]:
def append_all_words(preprocessed_docs):
    all_words=[]
    for doc in preprocessed_docs:
        for w in doc:
            all_words.append(w)
    return all_words

In [8]:
clus1_data = pd.read_csv(data_dir + 'Clus1.csv')

In [9]:
clus1_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5
0,"People love to get a new phone and, while rid...",1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",7-Mar-17,5,Open Honest Review,2,0.999757,3.290000e-18,2.830000e-10,2.390000e-09,0.000243
1,"Got it today as described It's very nice, runs...",1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",29-Sep-17,5,Excellent!,7,0.999952,9.120000e-16,8.430000e-11,8.020000e-09,0.000048
2,The os and speed is ok but do not drop it! I b...,1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",26-Mar-17,1,Good looking quick phone but don't drop it,10,0.752396,2.600000e-14,9.271750e-05,5.150000e-07,0.247511


In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

SIA = SentimentIntensityAnalyzer()

C:\Users\Rhea\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [11]:
clus1_data['sentiment_score'] = clus1_data['Review_Text'].apply(lambda x: SIA.polarity_scores(str(x))['compound'])

In [12]:
clus1_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5,sentiment_score
0,"People love to get a new phone and, while rid...",1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",7-Mar-17,5,Open Honest Review,2,0.999757,3.290000e-18,2.830000e-10,2.390000e-09,0.000243,0.9948
1,"Got it today as described It's very nice, runs...",1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",29-Sep-17,5,Excellent!,7,0.999952,9.120000e-16,8.430000e-11,8.020000e-09,0.000048,0.9570
2,The os and speed is ok but do not drop it! I b...,1,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",26-Mar-17,1,Good looking quick phone but don't drop it,10,0.752396,2.600000e-14,9.271750e-05,5.150000e-07,0.247511,0.8740


In [13]:
clus1_data['sentiment_score'].max()

0.99939999999999996

In [14]:
clus1_data['sentiment_score'].min()

-0.9042

In [15]:
clus1_data['sentiment_score'].mean()

0.32023636363636376

In [16]:
clus1_data.to_csv(data_dir + "Clus1_sentiment_score.csv")

In [17]:
clus1_data_samples= to_lower(clus1_data['Review_Text'].tolist())

In [18]:
tokenized_docs_cl1 =tokenize_docs(clus1_data_samples)
print(tokenized_docs_cl1[0])

['people', 'love', 'to', 'get', 'a', 'new', 'phone', 'and', ',', 'while', 'riding', 'that', 'new', 'tech', 'high', ',', 'get', 'on', 'amazon', 'and', 'shout', '``', 'best', 'phone', 'ever', "''", 'when', 'obviously', 'it', "'s", 'not', '.', 'here', 'is', 'my', 'open', 'honest', 'list', 'view', 'of', 'the', 'pros', 'and', 'cons', 'of', 'this', 'phone.pros', ':', 'price', ':', 'even', 'at', 'the', 'suggested', 'retail', 'of', '149', 'usd', ',', 'this', 'phone', 'out', 'performs', 'the', 'price', 'tagbuild', 'quality', ':', 'built', 'like', 'a', '700', 'usd', 'flagship', 'device', '.', 'manages', 'to', 'look', 'both', 'beautiful', '&', 'professionalcall', 'quality', ':', 'perfect', ',', 'which', 'means', 'it', 'does', 'what', 'a', 'phone', 'is', 'supposed', 'to', 'doscreen', ':', 'at', '720p', 'its', 'not', 'as', 'nice', 'as', 'an', 'iphone', 'or', 'pixel', ',', 'but', 'it', 'does', 'look', 'amazing', 'on', 'the', '5.5', 'inch', 'screenantenna', ':', 'most', 'forgotten', 'feature', 'of', 

In [29]:
clean_docs_cl1=remove_punctuation(tokenized_docs_cl1)

In [30]:
clean_docs_cl1=remove_stopwords(clean_docs_cl1)

In [31]:
clean_docs_cl1=lemmatise_tokens(clean_docs_cl1)
print(clean_docs_cl1)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [32]:
all_words_cl1=append_all_words(clean_docs_cl1)
print(len(all_words_cl1))

884147


In [33]:
frequency = {}
for word in all_words_cl1:
    count = frequency.get(word,0)
    frequency[word] = count + 1

In [34]:
frequency_list = frequency.keys()

In [37]:
uniq_words={}
for words in frequency_list:
    uniq_words[words]= frequency[words]
    print( words, frequency[words])

people 1026
love 2498
get 4347
new 2247
riding 325
tech 325
high 802
shout 325
best 3640
ever 3468
obviously 1406
open 778
honest 833
list 415
view 325
pro 1297
con 1429
phonepros 325
price 2009
even 5996
suggested 325
retail 587
149 975
usd 1625
performs 345
tagbuild 325
quality 2401
built 520
like 6722
700 784
flagship 2150
device 2377
manages 325
look 3084
beautiful 427
professionalcall 325
perfect 1110
mean 2437
supposed 1004
doscreen 325
720p 379
nice 3384
pixel 650
amazing 2662
55 325
inch 325
screenantenna 325
forgotten 325
feature 2037
review 3674
service 1558
basement 975
hospital 650
work 4070
tmobile 1153
duel 650
sims 743
decided 804
put 1312
personal 458
sim 7761
card 10313
sony 325
xperia 325
m5 325
ultra 379
got 2955
zero 361
part 553
vivo 3756
xl2 1697
least 878
2 1734
bar 344
4g 851
lte 1023
entire 411
radio 338
band 905
supported 325
yet 1144
signalsound 325
loud 361
crisp 325
back 3893
firing 325
speaker 1206
prefer 941
front 2012
facing 325
oh 325
wellcamera 325
gre

released 414
moth 328
predecessor 328
torn 328
trying 674
decide 328
acquire 328
considering 864
proved 1142
checking 328
hesitate 328
chose 328
expecting 628
previous 840
onehowever 328
terribly 382
wrong 417
fails 328
notâ€¢its 328
incredibly 656
sensitive 430
extent 328
reacts 328
touch 399
earphone 337
wire 328
listening 328
watching 328
closed 328
changed 462
earphonesâ€¢both 328
13mp 328
worst 656
primary 328
blurring 328
granulated 328
rough 328
lacking 328
brightness 328
secondary 328
decent 328
outside 529
direct 328
sunlight 328
snapped 328
indoors 328
usually 544
lesser 328
turn 1285
blurred 328
undefined 328
photography 328
vivid 328
sharp 437
nothing 1189
dramatic 328
â€¢the 328
continuously 656
freeze 664
crashâ€¢but 328
smartphone 1933
offer 947
compared 850
normal 909
single 342
careful 364
department 328
reach 328
15 328
counter 328
shut 328
happened 328
performing 328
guessed 328
happen 656
kept 418
alive 328
le 888
outstanding 696
exaggerationalso 328
point 978
remov

” 30
borrow 30
friend 30
fiveminute 30
messenger 38
ups 54
rid 38
opening 38
closing 38
pissed 38
hello 1
hikingmy 308
criticism 308
adoptable 924
heard 308
correct 308
itbut 308
sized 308
rhinoshield 308
crash 308
guardi 308
email 308
devicei 308
alltrails 308
pdf 308
tom 308
harrison 308
hiking 616
garmin 308
etrex 2772
white 308
vista 308
nuvi 308
3490 308
recording 924
track 924
juice 308
hike 308
record 308
heavily 616
forested 616
area 705
grief 308
whereas 308
utterly 308
topo 308
trounces 308
canon 308
powershot 308
s120 616
eye 308
stuff 308
image 687
sensor 308
optical 616
zoom 308
stabilization 308
twist 308
shoot 687
uhd 616
30fps 308
default 308
enormous 397
capacity 308
lower 308
priced 308
vacation 308
resisted 308
social 308
medium 308
texting 308
garbage 308
bill 308
samsung/tmobile 308
enable 308
featuresif 89
meant 89
stick 267
wall 89
surge 89
lying 89
flat 89
000 1
mention 13
misrepresentation 13
doe 52
snot 52
overlap 52
provide 123
densely 52
populated 52
markets

In [38]:
uniq_words['phonepros']

325

### CLUSTER 2

In [39]:
clus2_data = pd.read_csv(data_dir + 'Clus2.csv')

In [40]:
clus2_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5
0,LOVE that Amazon and BLU have teamed up! Absol...,2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",4-Dec-17,5,LOVE that Amazon and BLU have teamed up,92,4.370330e-04,0.873018,7.159627e-03,9.779160e-03,1.096064e-01
1,good product. I recommend,2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",23-Feb-18,4,Four Stars,129,5.080000e-12,0.965235,1.210000e-07,3.476499e-02,7.890000e-08
2,love it,2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",1-Mar-18,5,great,138,2.560000e-10,1.000000,4.060000e-11,2.810000e-08,1.150000e-09


In [41]:
clus2_data['sentiment_score'] = clus2_data['Review_Text'].apply(lambda x: SIA.polarity_scores(str(x))['compound'])

In [42]:
clus2_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5,sentiment_score
0,LOVE that Amazon and BLU have teamed up! Absol...,2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",4-Dec-17,5,LOVE that Amazon and BLU have teamed up,92,4.370330e-04,0.873018,7.159627e-03,9.779160e-03,1.096064e-01,0.9612
1,good product. I recommend,2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",23-Feb-18,4,Four Stars,129,5.080000e-12,0.965235,1.210000e-07,3.476499e-02,7.890000e-08,0.6597
2,love it,2,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",1-Mar-18,5,great,138,2.560000e-10,1.000000,4.060000e-11,2.810000e-08,1.150000e-09,0.6369


In [43]:
clus2_data['sentiment_score'].max()

0.9869

In [44]:
clus2_data['sentiment_score'].min()

-0.70020000000000004

In [45]:
clus2_data['sentiment_score'].mean()

0.5741163793103444

In [46]:
clus2_data.to_csv(data_dir + "Clus2_sentiment_score.csv")

### CLUSTER 3

In [47]:
clus3_data = pd.read_csv(data_dir + 'Clus3.csv')

In [48]:
clus3_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5
0,Everything works great only downside is the ea...,3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",6-Mar-17,4,Great Phone with a nice OS. Earpiece speaker i...,9,0.360637,9.930000e-09,0.600858,5.910649e-03,0.032595
1,I really like the phone. My only real problem...,3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",28-Apr-17,4,I really like the phone,12,0.000035,3.380000e-12,0.960885,2.460000e-09,0.039080
2,Loving this so far...lots of space and working...,3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",12-Nov-17,1,Something has gone wrong,14,0.000025,1.680000e-08,0.766267,1.340000e-09,0.233708


In [49]:
clus3_data['sentiment_score'] = clus3_data['Review_Text'].apply(lambda x: SIA.polarity_scores(str(x))['compound'])

In [50]:
clus3_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5,sentiment_score
0,Everything works great only downside is the ea...,3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",6-Mar-17,4,Great Phone with a nice OS. Earpiece speaker i...,9,0.360637,9.930000e-09,0.600858,5.910649e-03,0.032595,0.4019
1,I really like the phone. My only real problem...,3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",28-Apr-17,4,I really like the phone,12,0.000035,3.380000e-12,0.960885,2.460000e-09,0.039080,-0.8388
2,Loving this so far...lots of space and working...,3,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",12-Nov-17,1,Something has gone wrong,14,0.000025,1.680000e-08,0.766267,1.340000e-09,0.233708,0.8103


In [51]:
clus3_data['sentiment_score'].max()

0.98880000000000001

In [52]:
clus3_data['sentiment_score'].min()

-0.96550000000000002

In [53]:
clus3_data['sentiment_score'].mean()

0.06388627787307032

In [54]:
clus3_data.to_csv(data_dir + "Clus3_sentiment_score.csv")

### CLUSTER 4

In [55]:
clus4_data = pd.read_csv(data_dir + 'Clus4.csv')

In [56]:
clus4_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5
0,I have to give this 5 stars! For the price thi...,4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",25-Mar-17,5,Worth every penny!,6,7.080000e-06,3.004740e-05,1.290000e-07,0.999950,0.000013
1,Super smart phone. Veery cheap on price but ha...,4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",19-Feb-17,5,Great Smartphone on a Budget,16,2.080000e-07,5.690000e-07,2.060000e-08,0.999986,0.000014
2,"Bought one for my wife in April. Impressed, s...",4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",16-Aug-17,4,Love BLU,17,8.558810e-04,3.417950e-03,8.141508e-02,0.795476,0.118835


In [57]:
clus4_data['sentiment_score'] = clus4_data['Review_Text'].apply(lambda x: SIA.polarity_scores(str(x))['compound'])

In [58]:
clus4_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5,sentiment_score
0,I have to give this 5 stars! For the price thi...,4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",25-Mar-17,5,Worth every penny!,6,7.080000e-06,3.004740e-05,1.290000e-07,0.999950,0.000013,0.8898
1,Super smart phone. Veery cheap on price but ha...,4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",19-Feb-17,5,Great Smartphone on a Budget,16,2.080000e-07,5.690000e-07,2.060000e-08,0.999986,0.000014,0.9766
2,"Bought one for my wife in April. Impressed, s...",4,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",16-Aug-17,4,Love BLU,17,8.558810e-04,3.417950e-03,8.141508e-02,0.795476,0.118835,0.9464


In [59]:
clus4_data['sentiment_score'].max()

0.99509999999999998

In [60]:
clus4_data['sentiment_score'].min()

-0.80959999999999999

In [61]:
clus4_data['sentiment_score'].mean()

0.6395575824175829

In [62]:
clus4_data.to_csv(data_dir + "Clus4_sentiment_score.csv")

### CLUSTER 5

In [63]:
clus5_data = pd.read_csv(data_dir + 'Clus5.csv')

In [64]:
clus5_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5
0,"Over the years, I've tried maybe 6 Blu phones....",5,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",20-Feb-17,5,ABOUT FREAKING TIME!!!! UPDATED POST!!,1,0.000195,1.710000e-07,1.980000e-07,9.104930e-04,0.998894
1,Update seven months in: I've had to go from f...,5,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",30-May-17,1,Update - don't waste your money! A pretty sol...,3,0.116327,1.740000e-21,7.020000e-07,1.410000e-12,0.883672
2,"Still has a security issue,waiting on Blu to r...",5,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",3-Mar-17,1,Spyware still. Come on Blu get it together!,4,0.000003,3.940000e-21,5.648100e-04,2.100000e-11,0.999432


In [65]:
clus5_data['sentiment_score'] = clus5_data['Review_Text'].apply(lambda x: SIA.polarity_scores(str(x))['compound'])

In [66]:
clus5_data[0:3]

,Review_Text,Cluster_,Phone_Name,Date,Rating_Out_of_5,Review_Title,_document_,TextCluster_prob1,TextCluster_prob2,TextCluster_prob3,TextCluster_prob4,TextCluster_prob5,sentiment_score
0,"Over the years, I've tried maybe 6 Blu phones....",5,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",20-Feb-17,5,ABOUT FREAKING TIME!!!! UPDATED POST!!,1,0.000195,1.710000e-07,1.980000e-07,9.104930e-04,0.998894,0.9620
1,Update seven months in: I've had to go from f...,5,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",30-May-17,1,Update - don't waste your money! A pretty sol...,3,0.116327,1.740000e-21,7.020000e-07,1.410000e-12,0.883672,0.9825
2,"Still has a security issue,waiting on Blu to r...",5,"BLU VIVO XL2 - 5.5"" 4G LTE GSM",3-Mar-17,1,Spyware still. Come on Blu get it together!,4,0.000003,3.940000e-21,5.648100e-04,2.100000e-11,0.999432,0.7351


In [67]:
clus5_data['sentiment_score'].max()

0.99960000000000004

In [68]:
clus5_data['sentiment_score'].min()

-0.97729999999999995

In [69]:
clus5_data['sentiment_score'].mean()

0.26992558139534895

In [70]:
clus5_data.to_csv(data_dir + "Clus5_sentiment_score.csv")